# Homework Assignment 2: Data Prep
In this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset. Additionally, you will be asked to provide documentation for your functions.

## Step 1: Downloading the Data

This assignment will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1 we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

In this assignment, you will be utilizing a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contains many more extracted features than you were asked to compute for Homework 1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

__Note:__ Since the full dataset, and multiple copies of partially processed intermediary results, tend to take up a bit of space, you can use the toy dataset to implement and test your code. You may need to edit the data to fully test each of the requirements, but that is left as an exercise for the student. The full partition dataset is only included for those who wish to work with it once they have their code implemented. 

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [496]:
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np

### Read the dataset

In [497]:
data_dir = ''
data_file = 'partition1ExtractedFeatures.csv'

In [545]:
abt = pd.read_csv(os.path.join(data_dir, data_file), index_col=0)
print(abt.shape)

(73492, 820)


### Q1 (20 points)

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [546]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    
    """
    Description
    ----------
    Calculates information for a specific feature in a dataframe. 
    Information includes: the cardinality, count of null values, and other statistical information.
    
    Input
    ----------
    feature_name : str
        The name of the statistical feature 
    data : DataFrame
        The name of dataframe that  the feature belongs to.
     
    Output
    ----------
    Returns dataframe with the calculated information.
        
    """
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    
    dt = data[feature_name]
    rows = dt.shape[0]
    vals = []
    vals.append(feature_name)
    
    # cardinality:
    card = dt.nunique()
    vals.append(card)
    
    # non null count:
    nonnull_count = dt.notna().sum()
    vals.append(nonnull_count)
    # null count :
    null_count = dt.isna().sum()
    vals.append(null_count)
    
    # without nul values:
    dt_n = dt.dropna()
 
    # min: 
    mini = dt_n.min()
    vals.append(mini)
    
    # 1QR
    Q1 = np.percentile(dt_n, 25)
    vals.append(Q1)
    
    # mean:
    meani = np.mean(dt_n)
    vals.append(meani)
    
    # median:
    Q2 = np.percentile(dt_n, 50)
    vals.append(Q2)
    
    # 3QR
    Q3 = np.percentile(dt_n, 75)
    vals.append(Q3)
    
    # maximun:
    maxi = dt_n.max()
    vals.append(maxi)
    
    # std deviation
    std = np.std(dt_n)
    vals.append(std)
    
    # outliers count
    IQR = Q3 - Q1
    low_lim = Q1 - (1.5 * IQR)
    up_lim = Q3 + (1.5 * IQR)

    low_o = []
    up_o = []
    for x in dt_n:
        if (x < low_lim):
            low_o.append(x)
        elif (x > up_lim):
            up_o.append(x)
        else:
            continue
            
    low_count = len(low_o)
    vals.append(low_count)
    
    up_count = len(up_o)
    vals.append(up_count)
    
    final_arr = [summary_feature_names, vals]
    final_dt = pd.DataFrame(final_arr)
    
    return final_dt

In [547]:
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)
calc_summary_for('TOTUSJH_min', abt)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Feature Name,Cardinality,Non-null Count,Null Count,Min,25th,Mean,50th,75th,Max,Std. Dev,Outlier Count Low,Outlier Count High
1,TOTUSJH_min,30165,73492,0,0.0,29.756125,396.697968,110.011893,418.614528,5680.232811,694.293786,0,9014


### Q2 (20 points)
Using what you produced to answere Q1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [548]:
def construct_quality_report(data:DataFrame) -> DataFrame:
         
    """
    Description
    ----------
    Constructs data quality report of all the features in a dataset. Loops over all of the features
    and returns the information gotten from the calc_summary_for method.
    
    Input
    ----------
    data : DataFrame
        The name of dataframe for the dataset.
     
    Output
    ----------
    Returns new dataframe with the calculated information of all the features.
        
    """
    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']

    dt = data.drop(excluded_columns, axis=1)
    dtList = []
    for column in dt:
        dtList.append(calc_summary_for(column, dt).iloc[1].values.tolist())
        
    new_dt = pd.DataFrame(np.row_stack(dtList), columns= summary_feature_names)
    
    return new_dt

In [549]:
summary_table = construct_quality_report(abt)
print(summary_table.shape) 
summary_table

(816, 13)


,Feature Name,Cardinality,Non-null Count,Null Count,Min,25th,Mean,50th,75th,Max,Std. Dev,Outlier Count Low,Outlier Count High
0,TOTUSJH_min,30165,73492,0,0.0,29.75612495136377,396.69796846373106,110.01189278800209,418.6145279389365,5680.232810510071,694.293785755904,0,9014
1,TOTUSJH_max,24142,73492,0,0.7830986060844828,53.11711211132075,492.6490210164997,162.79736701329028,541.8758824650613,6078.23759924654,801.0783027224919,0,8540
2,TOTUSJH_median,62001,73492,0,0.0,40.72667088759274,439.41761034264476,133.98169262937176,472.71029070395946,5895.663838777607,741.1528873893251,0,8799
3,TOTUSJH_mean,73492,73492,0,0.1007557801706809,40.83318876176823,440.33338496214486,134.52413961554166,474.2575142389394,5877.701431801086,741.7674094925545,0,8783
4,TOTUSJH_stddev,73492,73492,0,0.1764714434391661,4.926720802584415,25.019193191617344,11.410789847231058,30.408329625988948,618.8349251489846,35.13048086854365,0,6684
...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,R_VALUE_last_value,33137,73492,0,0.0,0.0,1.600112878212864,0.0,3.5511780657611265,5.639195482297835,1.8711782239730768,0,0
812,R_VALUE_slope_of_longest_mono_increase,9737,73492,0,0.0,0.0,0.35039978190777726,0.0492121676402711,0.7523195917199933,1.734700358507385,0.4946065052249254,0,0
813,R_VALUE_slope_of_longest_mono_decrease,10383,73492,0,-1.6093862570385782,-0.00414125111335475,-0.051591060542203186,-0.0008510953699958,0.0,0.0,0.19537975280741127,10650,0
814,R_VALUE_avg_mono_increase_slope,40605,73492,0,0.0,0.0,0.1552462681203939,0.020343421899119453,0.2022604738457792,1.6367022980231436,0.26666586814293297,0,8287


### Q3 (20 points)
#### Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for Q2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [550]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    
    """
    Description
    ----------
    Drops features in dataset & summary table that have a cardinality of < 10.
    
    Input
    ----------
    summary_table : DataFrame
        dataframe through the method, construct_quality_report(data). 
    
    data : DataFrame
        dataframe of total dataset.
    Output
    ----------
    Returns the data dataframe without the features with a cardinality < 10.
        
    """
    summary_table['Cardinality']= pd.to_numeric(summary_table['Cardinality'])
    car = summary_table[summary_table['Cardinality'] < 10].index.tolist()
    summary_table.drop(car, inplace = True)
    dis = summary_table.iloc[car, 0]
    data.drop(dis, axis = 1, inplace = True)

In [551]:
drop_low_card_data(summary_table, abt)
print(summary_table.shape)

(801, 13)


### Q4 (20 points)
#### Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for Q2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for Q3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of `NaN` values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have **more than 1%** of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in Q3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [552]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    
    """
    Description
    ----------
    Drops features in dataset & summary table that have more than 1% Null/Nan entries.
    
    Input
    ----------
    summary_table : DataFrame
        dataframe through the method, construct_quality_report(data). 
    
    data : DataFrame
        dataframe of total dataset.
    Output
    ----------
    Returns the data dataframe without the features more than 1% Null/Nan entries.
        
    """
    
    to_drop = data.columns[data.isna().mean() > 0.01].tolist()
    data.drop(to_drop, axis = 1, inplace = True)
    
    indexNames = []
    for i in to_drop:
        indexNames.append(summary_table.index[summary_table['Feature Name']== i].tolist())
        
    flat_list = [item for sublist in indexNames for item in sublist]
    summary_table.drop(flat_list, inplace = True)
    

In [553]:
drop_excessive_nan_data(summary_table, abt)
print(summary_table.shape)

(799, 13)


Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next homework assignment.

In [554]:
# \data\MVTS
out_dir = ''
out_summary_table_name = 'data_quality_table.csv'
out_abt_name = 'cleaned_partition1ExtractedFeatures.csv'

In [ ]:
# Save here!
summary_path = os.path.join(out_dir, out_summary_table_name)
abt_path = os.path.join(out_dir, out_abt_name)

summary_table.to_csv(summary_path, sep="\t", index=False)
abt.to_csv(abt_path, sep="\t", index=False)

### Q5 (20 points)
#### Add docstring to your functions

Let's revisit our programming skill while learning Fundamentals of Data Science. 

Your code is only as valuable as its reusability. Without understandable and legible documentation (which makes maintainability and reusability possible) nobody would like to use your code, let alone to pay for it. ;)

If you want to know more about the value of documentation, read [this article](https://www.freecodecamp.org/news/why-documentation-matters-and-why-you-should-include-it-in-your-code-41ef62dd5c2f/). There are even conferences on this topic; see [this website](https://www.writethedocs.org/guide/writing/beginners-guide-to-docs/).

In Python, the documentation that is embedded in the code is called **docstring**. In the example below, the "string" wrapped in triple quotes is there to tell us all about this function.

In [ ]:
def nanmean(a, axis=None):
    """
    Compute the arithmetic mean along the specified axis, ignoring NaNs.
    
    Parameters
    ----------
    a : array_like
        Array containing numbers whose mean is desired. If `a` is not an
        array, a conversion is attempted.
    axis : {int, tuple of int, None}, optional
        Axis or axes along which the means are computed. The default is to compute
        the mean of the flattened array.
    """
    # some magic happens here that we don't care about.
    pass

Note that this is not just a *comment*. If you execute `nanmean` and then call it (as if you want to use it), you can hit `shift+Tab` while your cursor is on the function name, and see how the docstring gets compiled and then pops up. This allows other users to see our description even when they don't have access to our source code. Try it! You can do this with other NumPy and Pandas functions/methods that you've been using.

The above example is a simplified version of the method `nanmean` copied from the NumPy library ([here](https://github.com/numpy/numpy/blob/v1.21.0/numpy/lib/nanfunctions.py#L862-L957)). Feel free to check out their complete docstrings.


Your last task is to provide docstrings for the 4 methods you've implemented. Simply go back to those cells and modify your functions. Feel free to use the text provided to you (in the assignment descriptions) to enrich your docstrings. Keep in mind that your docstring needs (1) a general description, (2) a short description for each input, and (3) a short description for the output.

How to check your docstring? Hit `shift+Tab` and see if the pop-up message is correctly compiled, and make sure your description answers all the questions about your functions.